In [169]:
import numpy as np
import pandas as pd
import os

In [170]:
input_path = './training_data/'
output_path = './result_data/'

In [171]:
file_list = os.listdir(input_path)

In [172]:
file_list.sort()
file_list

['WA_TR_20191015.xlsx',
 'WA_TR_20191016.xlsx',
 'WA_TR_20191018.xlsx',
 'WA_TR_20191019.xlsx',
 'WA_TR_20191020.xlsx',
 'WA_TR_20191022.xlsx',
 'WA_TR_20191023.xlsx',
 'WA_TR_20191024.xlsx',
 'WA_TR_20191025.xlsx']

In [173]:
result_colmns = ['Name', 'Position', 'Date', 'Type_1', 'Type_2', 'Type_3', 'Height', 'Weight', 'RPE', 'TR_time_(min)', 'Total_dist_(m)', 'Dist_per_min_(m)', 'HSR+Sprint_(m)', 'MSR_(m)', 'HSR_(m)', 'Sprint_(m)', 'Accel_(m)', 'Decel_(m)', 'Max_speed_(km/h)', 'Load', 'Mono', 'Strain', 'EWAM']

In [174]:
def trainig_merge(file_name):
    file_path = input_path+file_name
    xl = pd.ExcelFile(file_path)
    
    sheet_num = len(xl.sheet_names)
    print('{} has {} sheets'.format(file_name, sheet_num))
    
    train_df = None
    
    if sheet_num==0:
        df = xl.parse(xl.sheet_names)
        df['Load'] = df['RPE']*df['TR_time_(min)']
        train_df = df        
    else:
        df_list = []
        for i, sn in enumerate(xl.sheet_names):
            print("Now try to merge Training Dataset : {} at {}".format(sn, file_path))
            df = xl.parse(sn)
            df['Load'] = df['RPE']*df['TR_time_(min)']
            if i == 0:
                train_df = df
            else:
                df_list.append(df)

        for df in df_list:
            train_df['Type'] = train_df['Type']+ ' / ' +df['Type']
    #         trian_df['Height'] = df['Height']
    #         trian_df['Weight'] = df['Weight']
            train_df['RPE'] = train_df['RPE'] + df['RPE'] 
            train_df['TR_time_(min)'] = train_df['TR_time_(min)'] + df['TR_time_(min)']
            train_df['Total_dist_(m)'] =  train_df['Total_dist_(m)'] + df['Total_dist_(m)']
            train_df['Dist_per_min_(m)'] = train_df['Dist_per_min_(m)'] + df['Dist_per_min_(m)']
            train_df['HSR+Sprint_(m)'] = train_df['HSR+Sprint_(m)'] + df['HSR+Sprint_(m)'] 
            train_df['MSR_(m)'] = train_df['MSR_(m)'] + df['MSR_(m)']
            train_df['HSR_(m)'] = train_df['HSR_(m)'] + df['HSR_(m)']
            train_df['Sprint_(m)'] = train_df['Sprint_(m)'] + df['Sprint_(m)']
            train_df['Accel_(m)'] = train_df['Accel_(m)'] + df['Accel_(m)']
            train_df['Decel_(m)'] = train_df['Decel_(m)'] + df['Decel_(m)']
            train_df['Max_speed_(km/h)'] = np.where((train_df['Max_speed_(km/h)'] >= df['Max_speed_(km/h)']), train_df['Max_speed_(km/h)'], df['Max_speed_(km/h)'])
            train_df['Load'] = train_df['Load'] + df['Load'] 

        # Average colums
        train_df['RPE'] = train_df['RPE'] / sheet_num
        train_df['Dist_per_min_(m)'] = train_df['Dist_per_min_(m)'] / sheet_num
    
    print('All Training Dataets were successfully merged.')
    return train_df

In [175]:
def result_by_player(train_df):
    
    day_df = None

    info_df = train_df.loc[0:0, 'Date':'Type']
    mean_df = train_df.loc[:,'Height':].mean().to_frame().T
    mean_df = info_df.join(mean_df)


    if os.path.isfile('./result_data/result_by_player.xlsx'):
        writer = pd.ExcelWriter('./result_data/result_by_player.xlsx')
        team_df = pd.read_excel('./result_data/result_by_player.xlsx', sheet_name='Team sheet')

        temp_df = team_df.iloc[:,:-3]
        temp_df = temp_df.append(mean_df, ignore_index=True)


        t_day = len(temp_df)

        if t_day <= 7:
            mean_df['Mono'] = temp_df['Load'].mean()/temp_df['Load'].std()
            mean_df['Strain'] = temp_df['Load'].sum()*mean_df['Mono']
            mean_df['EWAM'] = temp_df['Load'].sum()/temp_df['Load'].sum()
        elif t_day <=28:
            week_n = t_day - 7
            mean_df['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
            mean_df['Strain'] = temp_df.loc[week_n:,'Load'].sum()*mean_df['Mono']
            mean_df['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[week_n:,'Load'].sum()
        else:
            week_n = t_day - 7
            month_n = t_day - 28
            mean_df['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
            mean_df['Strain'] = temp_df.loc[week_n:,'Load'].sum()*mean_df['Mono']
            mean_df['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[month_n:,'Load'].sum()


        team_df = team_df.append(mean_df, ignore_index=True)
        team_df.to_excel(writer, sheet_name='Team sheet', columns=team_df.columns, index=False, float_format='%.2f')
        day_df = mean_df

        for i in range(len(train_df)):
            player = train_df.iloc[i].Name
            position = train_df.iloc[i].Position
            sheet_name = player+'_'+position
            player_data = train_df.iloc[i:i+1,3:]
            player_df = pd.read_excel('./result_data/result_by_player.xlsx', sheet_name=sheet_name)

            temp_df = player_df.iloc[:,:-3]
            temp_df = temp_df.append(player_data, ignore_index=True)

            t_day = len(temp_df)

            if t_day <= 7:
                player_data['Mono'] = temp_df['Load'].mean()/temp_df['Load'].std()
                player_data['Strain'] = temp_df['Load'].sum()*player_data['Mono']
                player_data['EWAM'] = temp_df['Load'].sum()/temp_df['Load'].sum()
            elif t_day <=28:
                week_n = t_day - 7
                player_data['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
                player_data['Strain'] = temp_df.loc[week_n:,'Load'].sum()*player_data['Mono']
                player_data['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[week_n:,'Load'].sum()
            else:
                week_n = t_day - 7
                month_n = t_day - 28
                player_data['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
                player_data['Strain'] = temp_df.loc[week_n:,'Load'].sum()*player_data['Mono']
                player_data['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[month_n:,'Load'].sum()

            player_df = player_df.append(player_data, ignore_index=True)
            player_df.to_excel(writer, sheet_name=sheet_name, index=False , float_format='%.2f')
            day_df = day_df.append(player_data, ignore_index=True)
            
        writer.save()

        print("We have result file. Add data on result file")
        print("We workout {} times.".format(t_day))
        print("========================================================================================")

    else:
        writer = pd.ExcelWriter('./result_data/result_by_player.xlsx')
        mean_df['Mono'] = mean_df['Load'].mean()/mean_df['Load'].std()
        mean_df['Strain'] = mean_df['Load'].sum()*mean_df['Mono']
        mean_df['EWAM'] = mean_df['Load'].sum()/mean_df['Load'].sum()
        mean_df.to_excel(writer, sheet_name='Team sheet', columns=mean_df.columns, index=False, float_format='%.2f')
        day_df = mean_df

        for i in range(len(df)):
            player = df.iloc[i].Name
            position = df.iloc[i].Position
            sheet_name = player+'_'+position
            player_data = df.iloc[i:i+1,3:]
            player_data['Mono'] = player_data['Load'].mean()/player_data['Load'].std()
            player_data['Strain'] = player_data['Load'].sum()*player_data['Mono']
            player_data['EWAM'] = player_data['Load'].sum()/player_data['Load'].sum()
            player_data.to_excel(writer, sheet_name=sheet_name, columns=player_data.columns, index=False, float_format='%.2f')
            day_df = day_df.append(player_data, ignore_index=True)

        writer.save()

        print("Result file doesn't exist. Make result file")
        print("We workout 1 time")
        print("========================================================================================")

    print("Complete!!!!")
    return day_df

In [176]:
tr_data = 'WA_TR_20191015.xlsx'
td = trainig_merge(tr_data)

WA_TR_20191015.xlsx has 2 sheets
Now try to merge Training Dataset : Sheet1 at ./training_data/WA_TR_20191015.xlsx
Now try to merge Training Dataset : Sheet2 at ./training_data/WA_TR_20191015.xlsx
All Training Dataets were successfully merged.


In [177]:
td.head()

,No.,Name,Position,Date,Type,Height,Weight,RPE,TR_time_(min),Total_dist_(m),Dist_per_min_(m),HSR+Sprint_(m),MSR_(m),HSR_(m),Sprint_(m),Accel_(m),Decel_(m),Max_speed_(km/h),Load
0,1,Kim min jung,GK,20191015,필드 훈련 / Match,175,57.2,32.10,69.0,11675.8,85.9,413.8,1434.4,377.8,36.0,10,10,28.2,533.2
1,2,Kang ga ae,GK,20191015,필드 훈련 / Match,172,54.5,30.75,101.0,16126.2,81.4,522.4,1615.2,423.6,98.8,10,10,28.3,802.0
2,3,Min yu kyeong,GK,20191015,필드 훈련 / Match,176,60.7,33.85,102.0,17883.8,90.3,838.0,2076.4,655.6,182.4,10,10,30.2,875.1
3,4,Lim seon joo,CB,20191015,필드 훈련 / Match,168,59.3,33.65,101.0,18222.2,92.0,649.0,2067.8,569.2,79.8,10,10,28.6,910.6
4,5,Hong hye ji,CB,20191015,필드 훈련 / Match,174,60.9,32.95,102.0,18506.6,93.5,191.8,1633.4,191.8,0.0,10,10,24.7,677.7


In [178]:
dd = result_by_player(td)

We have result file. Add data on result file
We workout 14 times.
Complete!!!!


In [193]:
def result_by_day(merged_data, day_result_data):
    train_day = str(day_result_data.iloc[0,0])
    team_day = day_result_data.iloc[0:1,:]    
    
    day_result_data = day_result_data.drop(0,0)
    day_df = merged_data.join(day_result_data.iloc[:,-3:])
    
    team_df = team_day.drop(columns=['Date', 'Type'])
    team_df = pd.DataFrame(team_df.values, columns = team_df.columns, index=['Team'])
    position_df = day_df.drop(columns = ['No.','Date']).groupby('Position').mean()    
    position_df = position_df.append(team_df)
    
    writer = pd.ExcelWriter('./result_data/result_by_day.xlsx')
    position_df.to_excel(writer, sheet_name=train_day, float_format='%.2f', startrow = 3, startcol=2)
    day_df.to_excel(writer, sheet_name=train_day, index=False, float_format='%.2f', startrow = 13)
    writer.save()   
    
    
    return position_df, day_df
    

In [196]:
position_df, day_df = result_by_day(td, dd)

In [197]:
day_df.head()

,No.,Name,Position,Date,Type,Height,Weight,RPE,TR_time_(min),Total_dist_(m),...,MSR_(m),HSR_(m),Sprint_(m),Accel_(m),Decel_(m),Max_speed_(km/h),Load,Mono,Strain,EWAM
0,1,Kim min jung,GK,20191015,필드 훈련 / Match,175,57.2,32.10,69.0,11675.8,...,1434.4,377.8,36.0,10,10,28.2,533.2,NaN,NaN,NaN
1,2,Kang ga ae,GK,20191015,필드 훈련 / Match,172,54.5,30.75,101.0,16126.2,...,1615.2,423.6,98.8,10,10,28.3,802.0,2.692349,8723.212027,1.0
2,3,Min yu kyeong,GK,20191015,필드 훈련 / Match,176,60.7,33.85,102.0,17883.8,...,2076.4,655.6,182.4,10,10,30.2,875.1,2.615538,12183.177887,1.0
3,4,Lim seon joo,CB,20191015,필드 훈련 / Match,168,59.3,33.65,101.0,18222.2,...,2067.8,569.2,79.8,10,10,28.6,910.6,2.764889,14094.020440,1.0
4,5,Hong hye ji,CB,20191015,필드 훈련 / Match,174,60.9,32.95,102.0,18506.6,...,1633.4,191.8,0.0,10,10,24.7,677.7,2.446227,12722.824470,1.0


In [192]:
position_df

,Height,Weight,RPE,TR_time_(min),Total_dist_(m),Dist_per_min_(m),HSR+Sprint_(m),MSR_(m),HSR_(m),Sprint_(m),Accel_(m),Decel_(m),Max_speed_(km/h),Load,Mono,Strain,EWAM
CB,170.000000,56.825000,31.787500,101.750000,18962.400000,95.750000,475.250000,2135.100000,435.900000,39.400000,10.0,10.0,28.000000,826.475000,3.064164,14847.117994,1.0
FW,161.000000,63.200000,36.100000,37.791209,3345.290110,84.870330,535.993407,259.670330,422.175824,113.815385,10.0,10.0,23.900000,502.720879,4.060012,15241.879216,1.0
GK,174.333333,57.466667,32.233333,90.666667,15228.600000,85.866667,591.400000,1708.666667,485.666667,105.733333,10.0,10.0,28.900000,736.766667,2.653944,10453.194957,1.0
MF,163.857143,55.242857,31.907143,59.697017,7741.556672,83.323862,438.287598,725.385400,362.202512,76.089011,10.0,10.0,24.978571,670.096075,3.386256,13320.692399,1.0
SB,164.250000,57.350000,33.175000,72.250000,13855.250000,100.450000,920.300000,1785.700000,754.850000,165.450000,10.0,10.0,29.350000,820.550000,3.575935,17168.942211,1.0
WF,164.000000,61.200000,34.475000,45.071429,5103.850549,85.878571,543.982418,477.909341,434.071429,109.911538,10.0,10.0,24.525000,513.855495,3.787752,13268.753068,1.0
Team,166.260870,57.556522,32.778261,69.737219,11082.956044,89.306593,571.145055,1219.921978,474.516484,96.638462,10.0,10.0,26.650000,697.705017,3.352604,14269.637243,1.0
